In [1]:
import os
from PIL import Image, ImageOps
from os.path import join
import numpy as np
import pandas as pd

In [2]:
img_path = "6937.bmp"

#### Transfer the img to B&W and to Binary metrix

In [3]:
def get_bin_table(threshold=240):
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
 
    return table

In [4]:
def trf_to_gry(img_path):
    image = Image.open(img_path)
    imgry = image.convert('L')
    table = get_bin_table()
    out = imgry.point(table, '1')
    return out

In [153]:
out = trf_to_gry(img_path)
out.save("BW","bmp")

In [6]:
img_bin_mtx = np.array(out)*1

In [7]:
sum(sum(np.array(out)*1))

292

#### Remove Noise Points

In [8]:
def clean_noise(np_mtx, corner_thshd, side_thshd, mid_thshd):
    mtx_height = np_mtx.shape[0]
    mtx_width = np_mtx.shape[1]
    
    np_mtx_new = np.zeros([mtx_height,mtx_width], dtype=int) # init am empty array to output later
    
    for i in range(mtx_height):
        for j in range(mtx_width):
            ### Handle Corner points
            if i == 0 and j == 0: # top left corner
                neibr_sum = 4-(np_mtx[i,j] + np_mtx[i+1,j] + np_mtx[i,j+1] + np_mtx[i+1,j+1])
                if neibr_sum > corner_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0

            elif i == 0 and j == mtx_width-1: # top right corner:
                neibr_sum = 4-(np_mtx[i,j] + np_mtx[i,j-1] + np_mtx[i+1,j] + np_mtx[i+1,j-1])
                if neibr_sum > corner_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            elif i == mtx_height -1 and j == 0: # bottom left corner:
                neibr_sum = 4-(np_mtx[i,j] + np_mtx[i,j+1] + np_mtx[i-1,j] + np_mtx[i-1,j+1])
                if neibr_sum > corner_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            elif i == mtx_height -1 and j == mtx_width-1: # bottom right corner:
                neibr_sum = 4-(np_mtx[i,j] + np_mtx[i,j-1] + np_mtx[i-1,j] + np_mtx[i-1,j-1])
                if neibr_sum > corner_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            ### Handle Edge
            elif i==0: # if top edge but not corner
                neibr_sum = 6-(np_mtx[i,j] + np_mtx[i,j-1] + np_mtx[i,j+1] \
                               + np_mtx[i+1,j-1] + np_mtx[i+1,j] + np_mtx[i+1,j+1])
                if neibr_sum > side_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            elif i==mtx_height -1: # if bottom edge but not corner
                neibr_sum = 6-(np_mtx[i,j] + np_mtx[i,j-1] + np_mtx[i,j+1] \
                               + np_mtx[i-1,j-1] + np_mtx[i-1,j] + np_mtx[i-1,j+1])
                if neibr_sum > side_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            elif j==0: # if left edge but not corner
                neibr_sum = 6-(np_mtx[i,j] + np_mtx[i-1,j] + np_mtx[i+1,j] \
                               + np_mtx[i-1,j+1] + np_mtx[i,j+1] + np_mtx[i+1,j+1])
                if neibr_sum > side_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            elif j==mtx_width-1: # if right edge but not corner
                neibr_sum = 6-(np_mtx[i,j] + np_mtx[i-1,j] + np_mtx[i+1,j] 
                               + np_mtx[i-1,j-1] + np_mtx[i,j-1] + np_mtx[i+1,j-1])
                if neibr_sum > side_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
            
            ### Handle middle points
            else:
                neibr_sum = 9-(np_mtx[i,j] \
                            + np_mtx[i-1,j-1] + np_mtx[i-1,j] + np_mtx[i-1,j+1] \
                            + np_mtx[i,j-1] + np_mtx[i,j+1] \
                            + np_mtx[i+1,j-1] + np_mtx[i+1,j] + np_mtx[i+1,j+1])
                if neibr_sum > mid_thshd and np_mtx[i,j]==0:
                    np_mtx_new[i,j] = 1
                else:
                    np_mtx_new[i,j] = 0
            
    return np_mtx_new

In [9]:
img_bin_mtx_clean = clean_noise(img_bin_mtx, 2, 2, 2)
sum(sum(img_bin_mtx_clean))

94

In [10]:
Image.fromarray(img_bin_mtx_clean==0)

#### Cut the image into seperate Characters

In [11]:
def digit_cut(cleaned_mtx):
    digit_list = []
    for digit_no in range(4):
        start_index = digit_no * 10 + 2
        end_index = digit_no * 10 + 8
        digit_list.append(cleaned_mtx[:,start_index:end_index+1])
    return digit_list

In [12]:
digit_list = digit_cut(img_bin_mtx_clean)

In [13]:
Image.fromarray(digit_list[0]==0)

In [14]:
Image.fromarray(digit_list[1]==0)

In [15]:
Image.fromarray(digit_list[2]==0)

In [16]:
Image.fromarray(digit_list[3]==0)

#### Feature Engineering

In [127]:
def trf_bin(dig_img_path):
    dig_img = Image.open(dig_img_path)
    dig_imgry = dig_img.convert('L')
    table = get_bin_table()
    dig_img_bw = dig_img.point(table, '1')
    
    dig_mtx = np.array(dig_img_bw)*1
    return dig_mtx

In [139]:
Image.fromarray(trf_bin("Train_data/2/0001-0.jpg")==1)

In [37]:
def ext_feature_bkup(dig_img_path):
    img_bin = trf_bin(dig_img_path)
    col0 = sum(img_bin)[0]
    col1 = sum(img_bin)[1]
    col2 = sum(img_bin)[2]
    col3 = sum(img_bin)[3]
    col4 = sum(img_bin)[4]
    col5 = sum(img_bin)[5]
    row0 = sum(img_bin[0])
    row1 = sum(img_bin[1])
    row2 = sum(img_bin[2])
    row3 = sum(img_bin[3])
    row4 = sum(img_bin[4])
    row5 = sum(img_bin[5])
    row6 = sum(img_bin[6])
    row7 = sum(img_bin[7])
    row8 = sum(img_bin[8])
    row9 = sum(img_bin[9])
    return [col0, col1, col2, col3, col4, col5
           ,row0, row1, row2, row3, row4, row5
            ,row6,row7,row8,row9]

In [38]:
ext_feature_bkup("Train_data/2/0001-0.jpg")

[6, 7, 7, 6, 6, 5, 2, 2, 4, 5, 5, 5, 5, 5, 4, 0]

In [30]:
def ext_feature(img_bin):
    col0 = sum(img_bin)[0]
    col1 = sum(img_bin)[1]
    col2 = sum(img_bin)[2]
    col3 = sum(img_bin)[3]
    col4 = sum(img_bin)[4]
    col5 = sum(img_bin)[5]
    row0 = sum(img_bin[0])
    row1 = sum(img_bin[1])
    row2 = sum(img_bin[2])
    row3 = sum(img_bin[3])
    row4 = sum(img_bin[4])
    row5 = sum(img_bin[5])
    row6 = sum(img_bin[6])
    row7 = sum(img_bin[7])
    row8 = sum(img_bin[8])
    row9 = sum(img_bin[9])
    return [col0, col1, col2, col3, col4, col5
           ,row0, row1, row2, row3, row4, row5
            ,row6,row7,row8,row9]

In [39]:
def create_train_data(dig_folder_path):
    df = pd.DataFrame(columns=['col0','col1','col2','col3','col4','col5'
                              ,'row0','row1','row2','row3','row4','row5'
                              ,'row6','row7','row8','row9'])
    img_list = os.listdir(dig_folder_path)
    for i in range(len(img_list)):
        img = img_list[i] 
        img_path = dig_folder_path+img
        df.loc[i] = ext_feature_bkup(img_path)
    return df

In [40]:
train_data = pd.DataFrame(columns=['col0','col1','col2','col3','col4','col5'
                              ,'row0','row1','row2','row3','row4','row5'
                              ,'row6','row7','row8','row9','digit'])
for digit in range(10):
    df_tmp = create_train_data("Train_data/"+str(digit)+"/")
    df_tmp["digit"] = str(digit)
    train_data = train_data.append(df_tmp)

In [41]:
train_data

,col0,col1,col2,col3,col4,col5,row0,row1,row2,row3,row4,row5,row6,row7,row8,row9,digit
0,2,7,4,3,8,2,2,4,4,1,2,2,2,3,4,2,0
1,1,7,3,3,7,2,2,3,3,2,2,2,1,3,4,1,0
2,2,7,4,4,8,2,2,4,4,1,2,2,2,4,4,2,0
3,2,8,4,4,8,2,2,4,4,2,2,2,2,4,4,2,0
4,2,8,4,4,7,2,2,4,4,2,1,2,2,4,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,6,7,7,7,7,4,3,4,4,4,3,2,5,5,5,3,9
156,6,7,7,7,6,4,3,4,4,4,3,1,5,5,5,3,9
157,6,6,6,6,7,3,3,3,4,4,3,2,3,4,5,3,9
158,5,6,7,7,6,4,3,4,4,4,2,2,4,4,5,3,9


#### Model Traning by Decision Tree

In [104]:
X = train_data.iloc[:,0:-1]
y = train_data.iloc[:,-1]

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [106]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(max_depth = 10)
dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [107]:
pred = dt_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(pred, y_test)

array([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 13,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 14,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 14,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 12,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 22,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  7]])

#### Model Traning by SVM

In [108]:
from sklearn.svm import SVC
svc_classifier = SVC()
svc_classifier.fit(X_train, y_train)

/Users/TonyWang/anaconda3/envs/cv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [155]:
pred = svc_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 13,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 14,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 15,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 12,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0, 22,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  7]])

In [156]:
accuracy_score(pred, y_test)

0.9913793103448276

#### Build Recognition Pipeline

In [151]:
def main_code_recog(img_path):
    img_gry = trf_to_gry(img_path) # transfer the image to B&W mode
    img_bin_mtx = np.array(img_gry)*1 # transfer the B&W img to matrix with 0&1
    img_bin_mtx_clean = 1- clean_noise(img_bin_mtx, 2, 2, 2) # noise points cleansing
    digit_list = digit_cut(img_bin_mtx_clean) # cut the image to seperate digits
    #init the test data
    feature_df = pd.DataFrame(columns=['col0','col1','col2','col3','col4','col5'
                              ,'row0','row1','row2','row3','row4','row5'
                              ,'row6','row7','row8','row9'])
    for i in range(len(digit_list)):
        digit_features = ext_feature(digit_list[i])
        feature_df.loc[i] = digit_features
    
    # pass the feature to the classifier
    pred = svc_classifier.predict(feature_df)
    return pred

In [152]:
main_code_recog("6937.bmp")

array(['6', '9', '3', '7'], dtype=object)